In [1]:
import qiskit as qk
import numpy as np
import math


In [2]:
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import GroverOperator

from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution

In [3]:
# Question: how to convert a function to an oracle?
def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

In [4]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)
oracle.draw()

┌───┐   ┌───┐
q_0: ──────■─┤ X ├─■─┤ X ├
           │ ├───┤ │ ├───┤
q_1: ──────■─┤ X ├─■─┤ X ├
     ┌───┐ │ ├───┤ │ └───┘
q_2: ┤ X ├─■─┤ X ├─■──────
     └───┘   └───┘

In [48]:
qasm3.dumps(MCMT(ZGate(), 3 - 1, 1))

'OPENQASM 3.0;\ninclude "stdgates.inc";\ngate mcphase_5418950080(_gate_p_0) _gate_q_0, _gate_q_1 {\n  cp(pi/2) _gate_q_0, _gate_q_1;\n}\ngate ccz _gate_q_0, _gate_q_1, _gate_q_2 {\n  cx _gate_q_0, _gate_q_2;\n  U(0, -pi/8, -pi/8) _gate_q_2;\n  cx _gate_q_1, _gate_q_2;\n  U(0, -7*pi/8, -7*pi/8) _gate_q_2;\n  cx _gate_q_0, _gate_q_2;\n  U(0, -pi/8, -pi/8) _gate_q_2;\n  cx _gate_q_1, _gate_q_2;\n  U(0, -7*pi/8, -7*pi/8) _gate_q_2;\n  mcphase_5418950080(pi/2) _gate_q_0, _gate_q_1;\n}\nqubit[3] q;\nccz q[0], q[1], q[2];\n'

In [45]:
qt = QuantumCircuit(1)
qt.z(0)
qt.draw()

┌───┐
q: ┤ Z ├
   └───┘

In [7]:
grover_op = GroverOperator(oracle)
grover_op.decompose().draw()

global phase: π
                 ┌───┐   ┌───┐┌───┐┌───┐     ┌───┐┌───┐     
state_0: ──────■─┤ X ├─■─┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────
               │ ├───┤ │ ├───┤├───┤├───┤  │  ├───┤├───┤     
state_1: ──────■─┤ X ├─■─┤ X ├┤ H ├┤ X ├──■──┤ X ├┤ H ├─────
         ┌───┐ │ ├───┤ │ ├───┤├───┤├───┤┌─┴─┐├───┤├───┤┌───┐
state_2: ┤ X ├─■─┤ X ├─■─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
         └───┘   └───┘   └───┘└───┘└───┘└───┘└───┘└───┘└───┘

In [8]:
optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

In [9]:
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()
qc.draw()

┌───┐┌────┐ ░ ┌─┐      
   q_0: ┤ H ├┤0   ├─░─┤M├──────
        ├───┤│    │ ░ └╥┘┌─┐   
   q_1: ┤ H ├┤1 Q ├─░──╫─┤M├───
        ├───┤│    │ ░  ║ └╥┘┌─┐
   q_2: ┤ H ├┤2   ├─░──╫──╫─┤M├
        └───┘└────┘ ░  ║  ║ └╥┘
meas: 3/═══════════════╩══╩══╩═
                       0  1  2

In [46]:

from qiskit import QuantumCircuit, qasm3, qasm2

In [58]:
qasm3.dumps(qc)

'OPENQASM 3.0;\ninclude "stdgates.inc";\ngate Q _gate_q_0 {\n  z _gate_q_0;\n  ry(-0.2) _gate_q_0;\n  x _gate_q_0;\n  z _gate_q_0;\n  x _gate_q_0;\n  ry(0.2) _gate_q_0;\n}\ngate Q_5234411664 _gate_q_0 {\n  Q _gate_q_0;\n}\nbit[1] meas;\nqubit[1] q;\nh q[0];\nQ_5234411664 q[0];\nbarrier q[0];\nmeas[0] = measure q[0];\n'

In [54]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [ ]:
# target = backend.target
# pm = generate_preset_pass_manager(target=target, optimization_level=3)

# circuit_isa = pm.run(qc)
# circuit_isa.draw(output="mpl", idle_wires=False, style="iqp")

In [18]:
import qiskit_aer
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.fake_provider import FakeVigo
from qiskit import transpile


In [19]:
backend = qiskit_aer.AerSimulator.from_backend(FakeVigo())

In [26]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

circuit_isa = pm.run(qc)
circuit_isa.draw()

»
ancilla_0 -> 0 ───────────────────────────────────────────────────────────────»
               ┌──────────┐┌────┐┌─────────┐ ┌───┐┌──────────┐┌───┐┌─────────┐»
      q_2 -> 1 ┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├»
               ├─────────┬┘├────┤├─────────┴┐└─┬─┘└──────────┘└─┬─┘└─────────┘»
      q_1 -> 2 ┤ Rz(π/2) ├─┤ √X ├┤ Rz(3π/4) ├──■────────────────┼─────────────»
               ├─────────┤ ├────┤├─────────┬┘                   │     ┌────┐  »
      q_0 -> 3 ┤ Rz(π/2) ├─┤ √X ├┤ Rz(π/2) ├────────────────────■─────┤ √X ├──»
               └─────────┘ └────┘└─────────┘                          └────┘  »
ancilla_1 -> 4 ───────────────────────────────────────────────────────────────»
                                                                              »
       meas: 3/═══════════════════════════════════════════════════════════════»
                                                                              »
«                                                                       »
«ancilla_0 -> 0 ────────────────────────────────────────────────────────»
«               ┌───┐┌───┐┌──────────┐┌───┐┌─────────┐             ┌───┐»
«      q_2 -> 1 ┤ X ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/2) ├─────────────┤ X ├»
«               └─┬─┘└───┘└──────────┘└─┬─┘└─────────┘             └─┬─┘»
«      q_1 -> 2 ──■─────────────────────┼────────────────────────────┼──»
«                                       │     ┌────┐  ┌───────────┐  │  »
«      q_0 -> 3 ────────────────────────■─────┤ √X ├──┤ Rz(-3π/2) ├──■──»
«                                             └────┘  └───────────┘     »
«ancilla_1 -> 4 ────────────────────────────────────────────────────────»
«                                                                       »
«       meas: 3/════════════════════════════════════════════════════════»
«                                                                       »
«                                                                            »
«ancilla_0 -> 0 ─────────────────────────────────────────────────────────────»
«               ┌────────┐   ┌────┐   ┌──────────┐                      ┌───┐»
«      q_2 -> 1 ┤ Rz(-π) ├───┤ √X ├───┤ Rz(-π/4) ├──■────────────────■──┤ X ├»
«               └────────┘   └────┘   └──────────┘┌─┴─┐┌──────────┐┌─┴─┐├───┤»
«      q_1 -> 2 ──────────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ X ├»
«                 ┌────┐  ┌──────────┐            └───┘└──────────┘└───┘└───┘»
«      q_0 -> 3 ──┤ √X ├──┤ Rz(-π/4) ├───────────────────────────────────────»
«                 └────┘  └──────────┘                                       »
«ancilla_1 -> 4 ─────────────────────────────────────────────────────────────»
«                                                                            »
«       meas: 3/═════════════════════════════════════════════════════════════»
«                                                                            »
«                                                                         »
«ancilla_0 -> 0 ──────────────────────────────────────────────────────────»
«               ┌───┐     ┌───┐┌───┐┌──────────┐┌───┐┌─────────┐  ┌───┐   »
«      q_2 -> 1 ┤ X ├──■──┤ X ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├──┤ X ├───»
«               └─┬─┘  │  └─┬─┘└─┬─┘└──────────┘└─┬─┘└─────────┘  └─┬─┘   »
«      q_1 -> 2 ──┼────┼────┼────■────────────────┼─────────────────■─────»
«                 │  ┌─┴─┐  │                     │     ┌────┐  ┌────────┐»
«      q_0 -> 3 ──■──┤ X ├──■─────────────────────■─────┤ √X ├──┤ Rz(-π) ├»
«                    └───┘                              └────┘  └────────┘»
«ancilla_1 -> 4 ──────────────────────────────────────────────────────────»
«                                                                         »
«       meas: 3/══════════════════════════════════════════════════════════»
«                                                                         »
«                                                            

In [20]:
tqc = transpile(qc, backend, scheduling_method="alap")

In [25]:
tqc.draw()

global phase: 9π/8
               ┌──────────────────┐                                 »
ancilla_0 -> 0 ┤ Delay(86768[dt]) ├─────────────────────────────────»
               └───┬──────────┬───┘   ┌────┐  ┌─────────┐   ┌───┐   »
      q_2 -> 1 ────┤ Rz(-π/2) ├───────┤ √X ├──┤ Rz(π/2) ├───┤ X ├───»
                   ├─────────┬┘       ├────┤  ├─────────┤   └─┬─┘   »
      q_1 -> 2 ────┤ Rz(π/2) ├────────┤ √X ├──┤ Rz(π/2) ├─────■─────»
               ┌───┴─────────┴───┐ ┌──┴────┴─┐└──┬────┬─┘┌─────────┐»
      q_0 -> 3 ┤ Delay(1184[dt]) ├─┤ Rz(π/2) ├───┤ √X ├──┤ Rz(π/2) ├»
               ├─────────────────┴┐└─────────┘   └────┘  └─────────┘»
ancilla_1 -> 4 ┤ Delay(86768[dt]) ├─────────────────────────────────»
               └──────────────────┘                                 »
       meas: 3/═════════════════════════════════════════════════════»
                                                                    »
«                                                               »
«ancilla_0 -> 0 ────────────────────────────────────────────────»
«                   ┌──────────┐   ┌───┐    ┌─────────┐    ┌───┐»
«      q_2 -> 1 ────┤ Rz(-π/4) ├───┤ X ├────┤ Rz(π/4) ├────┤ X ├»
«               ┌───┴──────────┴──┐└─┬─┘    └─────────┘    └─┬─┘»
«      q_1 -> 2 ┤ Delay(2080[dt]) ├──┼───────────────────────■──»
«               └─────────────────┘  │  ┌─────────────────┐     »
«      q_0 -> 3 ─────────────────────■──┤ Delay(1184[dt]) ├─────»
«                                       └─────────────────┘     »
«ancilla_1 -> 4 ────────────────────────────────────────────────»
«                                                               »
«       meas: 3/════════════════════════════════════════════════»
«                                                               »
«                                                                     »
«ancilla_0 -> 0 ──────────────────────────────────────────────────────»
«                   ┌──────────┐              ┌───┐       ┌───┐       »
«      q_2 -> 1 ────┤ Rz(-π/4) ├──────────────┤ X ├───────┤ X ├───────»
«               ┌───┴──────────┴──┐┌─────────┐└─┬─┘       └───┘       »
«      q_1 -> 2 ┤ Delay(2240[dt]) ├┤ Rz(π/4) ├──┼─────────────────────»
«               └─────────────────┘└─────────┘  │  ┌─────────────────┐»
«      q_0 -> 3 ────────────────────────────────■──┤ Delay(3552[dt]) ├»
«                                                  └─────────────────┘»
«ancilla_1 -> 4 ──────────────────────────────────────────────────────»
«                                                                     »
«       meas: 3/══════════════════════════════════════════════════════»
«                                                                     »
«                                                                              »
«ancilla_0 -> 0 ───────────────────────────────────────────────────────────────»
«               ┌──────────┐┌───┐     ┌───┐                   ┌───┐┌──────────┐»
«      q_2 -> 1 ┤ Rz(-π/4) ├┤ X ├──■──┤ X ├───────────────────┤ X ├┤ Rz(-π/4) ├»
«               └──────────┘└─┬─┘┌─┴─┐└─┬─┘┌─────────────────┐└─┬─┘└──────────┘»
«      q_1 -> 2 ──────────────■──┤ X ├──■──┤ Delay(4320[dt]) ├──┼──────────────»
«                                └───┘     └─────────────────┘  │  ┌─────────┐ »
«      q_0 -> 3 ────────────────────────────────────────────────■──┤ Rz(π/4) ├─»
«                                                                  └─────────┘ »
«ancilla_1 -> 4 ───────────────────────────────────────────────────────────────»
«                                                                              »
«       meas: 3/═══════════════════════════════════════════════════════════════»
«                                                                              »
«                                                                    »
«ancilla_0 -> 0 ─────────────────────────────────────────────────────»
«               ┌───┐       ┌───┐                               ┌───┐»
«      q_2 

In [27]:
res = backend.run(circuit_isa)

In [31]:
res.result()

Result(backend_name='aer_simulator_from(fake_vigo)', backend_version='1.3.6', qobj_id='', job_id='cffee0b3-9c59-4163-ab40-f02a5ea62583', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x5': 30, '0x6': 47, '0x1': 52, '0x4': 385, '0x0': 60, '0x2': 41, '0x7': 44, '0x3': 365}), header=QobjExperimentHeader(creg_sizes=[['meas', 3]], global_phase=0.0, memory_slots=3, n_qubits=5, name='circuit-1282', qreg_sizes=[['q', 5]], metadata={}), status=DONE, seed_simulator=1723122494, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 10, 'parallel_shots': 1, 'sample_measure_time': 0.00012425, 'noise': 'superop', 'batched_shots_optimization': False, 'remapped_qubits': True, 'active_input_qubits': [1, 2, 3], 'device': 'CPU', 'time_taken': 0.002130791, 'measure_sampling': True, 'num_clbits': 3, 'max_memory_mb': 32768, 'input_qubit_map': [[3, 2], [2, 1], [1, 0]], 'num_qubits': 3, 'method': 'density_m

In [33]:
res.result().get_counts()

{'101': 30,
 '110': 47,
 '001': 52,
 '100': 385,
 '000': 60,
 '010': 41,
 '111': 44,
 '011': 365}

In [32]:
plot_histogram(res.result().get_counts(), title="Grover/Z")

MissingOptionalLibraryError: "The 'matplotlib' library is required to use '_plotting_core'. You can install it with 'pip install matplotlib'."